In [1]:
import pandas as pd
from utils_analysis import prepare_scores
import matplotlib.pyplot as plt
import numpy as np
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr
from scipy.spatial.distance import jensenshannon
import json
from src.eval import calculate_bleu
from src.eval import calculate_rouge_single

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
df = pd.read_csv("data/flores/test.csv")

# "results/bart/r1/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r2/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r3/bart-base_data_flores_checkpoint-1000_flores.json",
# "results/t5/r1/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r2/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r3/flan-t5-base_data_flores_checkpoint-1000_flores.json",

results = prepare_scores(
    "results/flan-t5-base_data_flores_checkpoint-1000_flores.json",
    df["target"],
    "bleu",
)

baseline_probs = json.load(
    open(
        "results/t5_baseline/google_flan-t5-base_flores.json",
        "r",
    )
)

float division by zero
float division by zero
float division by zero


/home/mila/f/floresl/beam-search/utils_analysis.py:45: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [6]:
beam_score_log_probs = pd.DataFrame(results.scores_by_beam["beam_score_log_probs"])

In [7]:
results.scores_dataframe.drop(["sentences", "dropout_sentences"], axis=1).corr(
    method="spearman"
)["bleu"]

length_normalized_log_probs    0.080417
mean_token_entropy            -0.069656
dropout_bleu_variance          0.407076
dropout_meteor_score           0.477018
dropout_entropy               -0.226441
dropout_disagreement           0.055734
bleu                           1.000000
beam_score_ratios_99           0.222818
beam_score_log_probs_99        0.020158
beam_score_top_k_99            0.050499
beam_score_impt_wt_99         -0.050880
Name: bleu, dtype: float64

### Tail Index

In [8]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=1)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, results.scores_dataframe["bleu"])

SignificanceResult(statistic=0.33566514658213226, pvalue=4.499744944356827e-28)

### Baseline Metrics

In [9]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]
baseline_log_probs = pd.DataFrame(baseline_log_probs).values

beam_score_log_probs = beam_score_log_probs.values

In [10]:
js_distances = []
for i in range(1012):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1),
        softmax(baseline_log_probs[i], temperature=1),
    )
    js_distances.append(js)
spearmanr(js_distances, results.scores_dataframe["bleu"])

SignificanceResult(statistic=0.31346563399159383, pvalue=1.6426005234166847e-24)

In [11]:
js_distances = []
for i in range(1012):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1), np.array([1 / 100] * 100)
    )
    js_distances.append(js)
spearmanr(js_distances, results.scores_dataframe["bleu"])

SignificanceResult(statistic=0.3362153416100313, pvalue=3.639991728735621e-28)

### Oracle Methods

In [12]:
beam_search_sentences = results.sentences

In [13]:
def get_list_of_scores(target: str, predictions: list[str]):
    return list(
        map(
            lambda pred: calculate_bleu([pred], [[target]]),
            predictions,
        )
    )

In [14]:
weighted_avgs = list[float]()
corr_w_scores = list[float]()
for i in range(1012):
    quality_scores = get_list_of_scores(df["target"][i], results.sentences[i])
    weighted_avg = (
        float(np.average(beam_score_log_probs[i], weights=quality_scores))
        if sum(quality_scores) > 0
        else 0
    )
    corr_w_score = spearmanr(beam_score_log_probs[i], quality_scores).statistic
    if str(corr_w_score) == "nan":
        corr_w_score = 0.0
    weighted_avgs.append(weighted_avg)
    corr_w_scores.append(corr_w_score)

/tmp/ipykernel_1239057/3743040825.py:10: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_w_score = spearmanr(beam_score_log_probs[i], quality_scores).statistic


float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float divis

In [15]:
spearmanr(weighted_avgs, results.scores_dataframe["bleu"])

SignificanceResult(statistic=0.05150370214418871, pvalue=0.10152708135420219)

In [16]:
spearmanr(corr_w_scores, results.scores_dataframe["bleu"])

SignificanceResult(statistic=0.19976456295383235, pvalue=1.4383123295231031e-10)